In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [3]:

X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
# 
# future thinking here...
# ..disk approach => return the location of X, y on disk instead 

In [4]:
fu.get_my_memory()

{'pmem': '26.9', 'rss': '0.525 GiB'}

In [ ]:
%%time
reload(pv2)
# wow kernel keeps dying here .. guess got to baatch the preprocessing.
#with ipdb.launch_ipdb_on_exception():
X_transformed, y_enc, proc_dict = pv2.preprocess(
        X, y, neighborhoods, labeled=True)

 64%|██████▎   | 7/11 [00:11<00:09,  2.30s/it]

- hmm strange bug?..  from my ipdb runcall..
- "the array at index 0 has size     84341" 
- "and the array at index 1 has size 843416"
- 
```python
ipdb> 
> /opt/program/fresh/preproc/v2.py(55)xform()
     54                 ),
---> 55             X[a:b, 4:5]
     56             ))

ipdb> 
ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 84341 and the array at index 1 has size 843416
> /opt/program/fresh/preproc/v2.py(55)xform()
     54                 ),
---> 55             X[a:b, 4:5]
     56             ))
```

#### planb?
- If i cant get this transformation partitioned in memory... 
- then plan b should be going deeper on the disk-cache numpy approach.

#### plan b indeed.
- So after partitioning, getting  half way through and dead kernel again

```
 64%|██████▎   | 7/11 [00:11<00:09,  2.30s/it]
```
- so yea, I'm thinking either trying to run this as a separate process, 
- or yea use h5py , or native numpy disk based storage here..
- and in general this means the processed data cannot be in memory anyway, so for sure going to have to find some way to stitch up the data on disk w/ the numpy / h5py . 

In [7]:
import fresh.utils as fu
from importlib import reload
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [6]:
num_rows = 1000
slices = fu.get_slices(list(range(num_rows)), num_slices=10)
for a,b in slices:
    print(a)

0
100
200
300
400
500
600
700
800
900


In [2]:
#fu.get_my_memory()#
!top

=top - 13:19:19 up 6 days,  4:33,  0 users,  load average: 0.59, 0.29, 0.14
Tasks:   5 total,   1 running,   4 sleeping,   0 stopped,   0 zombie
%Cpu(s):  7.1 us,  0.6 sy,  0.0 ni, 92.1 id,  0.2 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem :  2046844 total,  1820120 free,   111344 used,   115380 buff/cache
KiB Swap:  1048572 total,   776864 free,   271708 used.  1791560 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
 1296 root      20   0  562408  52916  13632 S   6.7  2.6   0:00.30 python      
    1 root      20   0   18236     40      0 S   0.0  0.0   0:00.22 bash        
 1053 root      20   0  558948  19272   6204 S   0.0  0.9   0:18.98 jupyter-no+ 
 1310 root      20   0    4500    748    676 S   0.0  0.0   0:00.52 sh          
 1311 root      20   0   36852   3076   2668 R   0.0  0.2   0:00.00 top         
>
